In [12]:
# Dependencies and Setup
import requests
import json
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import time
import os
import csv

# Import the API key
from api_key import bls_api_key

# BLS seriesIDs for the API
Source: United States Department of Labor: BLS Beta Labs

United States of America: National Unemployment Rate
1. LNS14000000: '(Seas) Unemployment Rate'
2. LNU04000000: 'Unadjusted Unemployment Rate'

United States of America: Unemployment Rate, Seasonally Adjusted by State
1. Alabama: Series ID LASST010000000000003 - Unemployment Rate: Alabama (S)
2. Alaska: Series ID  LASST020000000000003 - Unemployment Rate: Alaska (S)
3. Arizona: Series ID LASST040000000000003 - Unemployment Rate: Arizona (S)
4. Arkansas: Series ID    LASST050000000000003 - Unemployment Rate: Arkansas (S)
5. California: Series ID  LASST060000000000003 - Unemployment Rate: California (S)
6. Colorado: Series ID    LASST080000000000003 - Unemployment Rate: Colorado (S)
7. Connecticut: Series ID LASST090000000000003 - Unemployment Rate: Connecticut (S)
8. Delaware: Series ID    LASST100000000000003 - Unemployment Rate: Delaware (S)
9. Florida: Series ID LASST120000000000003 - Unemployment Rate: Florida (S)
10. Georgia: Series ID    LASST130000000000003 - Unemployment Rate: Georgia (S)
11. Hawaii: Series ID LASST150000000000003 - Unemployment Rate: Hawaii (S)
12. Idaho: Series ID  LASST160000000000003 - Unemployment Rate: Idaho (S)
13. Illinois: Series ID   LASST170000000000003 - Unemployment Rate: Illinois (S)
14. Indiana: Series ID LASST180000000000003 - Unemployment Rate: Indiana (S)
15. Iowa: Series ID   LASST190000000000003 - Unemployment Rate: Iowa (S)
16. Kansas: Series ID LASST200000000000003 - Unemployment Rate: Kansas (S)
17. Kentucky: Series ID   LASST210000000000003 - Unemployment Rate: Kentucky (S)
18. Louisiana: Series ID  LASST220000000000003 - Unemployment Rate: Louisiana (S)
19. Maine: Series ID  LASST230000000000003 - Unemployment Rate: Maine (S)
20. Maryland: Series ID   LASST240000000000003 - Unemployment Rate: Maryland (S)
21. Massachusetts: Series ID  LASST250000000000003 - Unemployment Rate: Massachusetts (S)
22. Michigan: Series ID   LASST260000000000003 - Unemployment Rate: Michigan (S)
23. Minnesota: Series ID  LASST270000000000003 - Unemployment Rate: Minnesota (S)
24. Mississippi: Series ID    LASST280000000000003 - Unemployment Rate: Mississippi (S)
25. Missouri: Series ID   LASST290000000000003 - Unemployment Rate: Missouri (S)
26. Montana: Series ID    LASST300000000000003 - Unemployment Rate: Montana (S)
27. Nebraska: Series ID   LASST310000000000003 - Unemployment Rate: Nebraska (S)
28. Nevada: Series ID LASST320000000000003 - Unemployment Rate: Nevada (S)
29. New Hampshire: Series ID  LASST330000000000003 - Unemployment Rate: New Hampshire (S)
30. New Jersey: Series ID LASST340000000000003 - Unemployment Rate: New Jersey (S)
31. New Mexico: Series ID LASST350000000000003 - Unemployment Rate: New Mexico (S)
32. New York: Series ID   LASST360000000000003 - Unemployment Rate: New York (S)
33. North Carolina: Series ID LASST370000000000003 - Unemployment Rate: North Carolina (S)
34. North Dakota: Series ID   LASST380000000000003 - Unemployment Rate: North Dakota (S)
35. Ohio: Series ID   LASST390000000000003 - Unemployment Rate: Ohio (S)
36. Oklahoma: Series ID   LASST400000000000003 - Unemployment Rate: Oklahoma (S)
37. Oregon: Series ID LASST410000000000003 - Unemployment Rate: Oregon (S)
38. Pennsylvania: Series ID   LASST420000000000003 - Unemployment Rate: Pennsylvania (S)
39. Rhode Island: Series ID   LASST440000000000003 - Unemployment Rate: Rhode Island (S)
40. South Carolina: Series ID LASST450000000000003 - Unemployment Rate: South Carolina (S)
41. South Dakota: Series ID   LASST460000000000003 - Unemployment Rate: South Dakota (S)
42. Tennessee: Series ID  LASST470000000000003 - Unemployment Rate: Tennessee (S)
43. Texas: Series ID  LASST480000000000003 - Unemployment Rate: Texas (S)
44. Utah: Series ID   LASST490000000000003 - Unemployment Rate: Utah (S)
45. Vermont: Series ID    LASST500000000000003 - Unemployment Rate: Vermont (S)
46. Virginia: Series ID   LASST510000000000003 - Unemployment Rate: Virginia (S)
47. Washington: Series ID LASST530000000000003 - Unemployment Rate: Washington (S)
48. West Virginia: Series ID  LASST540000000000003 - Unemployment Rate: West Virginia (S)
49. Wisconsin: Series ID  LASST550000000000003 - Unemployment Rate: Wisconsin (S)
50. Wyoming: Series ID LASST560000000000003 - Unemployment Rate: Wyoming (S)
Bonus
51. Puerto Rico: Series ID    LASST720000000000003 - Unemployment Rate: Puerto Rico (S)
52. District of Columbia: Series ID   LASST110000000000003 - Unemployment Rate: District of Columbia (S)

United States of America: Unemployment Rate, (Seasonally Adjusted) by Industry
(We are using Nonagricultural Private Wage and Salary Workers for the industry unemployment rates)
1. Healthcare
2. Hospitality
3. Technology

    'LNU04032231': 'Construction Industry',
    'LNU04034236': 'Educational Services',
    'LNU04034239': 'Health Care and Social Assistance',
    'LNU04032230': 'Mining, quarrying, and oil and gas extraction',
    'LNU04034202': 'Finance',
    'LNU04034211': 'Real Estate',
    'LNU04034205': 'Insurance',
    'LNU04034199': 'Finance and Insurance'

We will not be using seriesID that ends with Q, which stands for Quarter

# BLS API Version 1 Known Limitations
1. Users must have knowledge of BLS Series IDs to successfully complete a request.
2. The API will only return observational values and footnotes. It does not return metadata.
3. There is a maximum of 25 time series in a single request.
4. Users may request a maximum of 500 queries daily.
5. Users may request a maximum of 20 years of data.

Due to limitation of 25 time series in a single request, the API request will be broken up into 3 separate requests then merged later

# BLS Request 1


In [13]:

# Create the URL to retrieve the data
url = 'https://api.bls.gov/publicAPI/v2/timeseries/data/'
# api_key.py file contains: bls_api_key = 'key'
key = '?registrationkey={}'.format(bls_api_key)
# Series stored as a dictionary
series_dict = {
    'LNS14000000': 'Unemployment Rate (S)',
    'LNU04000000': 'Unemployment Rate (U)',
    'LNS14000002': 'Women (S)',
    'LNS14000001': 'Men (S)',
    'LNS14200000': 'Part-Time Workers (S)',
    'LNS14100000': 'Full-Time Workers (S)',
    'LNU04032241': 'Leisure and Hospitality (U)',
    'LNU04032240': 'Education and Health Services (U)',
    'LNU04032238': 'Financial Activities Industry (U)',
    'LNU04032235': 'Wholesale and Retail Trade (U)',
    'LASST110000000000003': 'District of Columbia (S)'}

#    'LASST720000000000003': 'Puerto Rico (S)' <-- NOT WORKING - ERROR TYPE

# Years
dates = ('2018', '2019', '2020', '2021', '2022')

In [14]:
# JSON request using the URL with the seriesID and dates
headers = {'Content-type': 'application/json'}
# Submit the list of series as data
data = json.dumps({
    "seriesid": list(series_dict.keys()),
    "startyear": dates[0],
    "endyear": dates[4]})
# Post request for the data
results = requests.post(
    '{}{}'.format(url, key),
    headers=headers,
    data=data).json()['Results']['series']

#print(p)

KeyError: 'series'

In [ ]:
# Date index from first series
date_list = [f"{i['year']}-{i['period'][1:]}-01" for i in results[0]['data']]
# Empty dataframe to fill with values
unemployment1_df = pd.DataFrame()
# Build a pandas series from the API results, p
for s in results:
    unemployment1_df[series_dict[s['seriesID']]] = pd.Series(
        index = pd.to_datetime(date_list),
        data = [i['value'] for i in s['data']]
        ).astype(float).iloc[::-1]
# Show last 5 results
unemployment1_df.tail()

# BLS Request No. 2

In [ ]:

# Create the URL to retrieve the data
url = 'https://api.bls.gov/publicAPI/v2/timeseries/data/'
# api_key.py file contains: bls_api_key = 'key'
key = '?registrationkey={}'.format(bls_api_key)
# Series stored as a dictionary
series_dict = {
    'LASST010000000000003': 'Alabama (S)',
    'LASST020000000000003': 'Alaska (S)',
    'LASST040000000000003': 'Arizona (S)',
    'LASST050000000000003': 'Arkansas (S)',
    'LASST060000000000003': 'California (S)',
    'LASST080000000000003': 'Colorado (S)',
    'LASST090000000000003': 'Connecticut (S)',
    'LASST100000000000003': 'Delaware (S)',
    'LASST120000000000003': 'Florida (S)',
    'LASST130000000000003': 'Georgia (S)',
    'LASST150000000000003': 'Hawaii (S)',
    'LASST160000000000003': 'Idaho (S)',
    'LASST170000000000003': 'Illinois (S)',
    'LASST180000000000003': 'Indiana (S)',
    'LASST190000000000003': 'Iowa (S)',
    'LASST200000000000003': 'Kansas (S)',
    'LASST210000000000003': 'Kentucky (S)',
    'LASST220000000000003': 'Louisiana (S)',
    'LASST230000000000003': 'Maine (S)',
    'LASST240000000000003': 'Maryland (S)',
    'LASST250000000000003': 'Massachusetts (S)',
    'LASST260000000000003': 'Michigan (S)',
    'LASST270000000000003': 'Minnesota (S)',
    'LASST280000000000003': 'Mississippi (S)',
    'LASST290000000000003': 'Missouri (S)'}
# Years
dates = ('2018', '2019', '2020', '2021', '2022')

In [ ]:
# JSON request using the URL with the seriesID and dates
headers = {'Content-type': 'application/json'}
# Submit the list of series as data
data2 = json.dumps({
    "seriesid": list(series_dict.keys()),
    "startyear": dates[0],
    "endyear": dates[4]})
# Post request for the data
results2 = requests.post(
    '{}{}'.format(url, key),
    headers=headers,
    data=data2).json()['Results']['series']


In [ ]:
# Date index from first series
date_list = [f"{i['year']}-{i['period'][1:]}-01" for i in results2[0]['data']]
# Empty dataframe to fill with values
unemployment2_df = pd.DataFrame()
# Build a pandas series from the API results, p
for s in results2:
    unemployment2_df[series_dict[s['seriesID']]] = pd.Series(
        index = pd.to_datetime(date_list),
        data = [i['value'] for i in s['data']]
        ).astype(float).iloc[::-1]
# Show last 5 results
unemployment2_df.tail()

# BLS Request No. 3

In [ ]:

# Create the URL to retrieve the data
url = 'https://api.bls.gov/publicAPI/v2/timeseries/data/'
# api_key.py file contains: bls_api_key = 'key'
key = '?registrationkey={}'.format(bls_api_key)
# Series stored as a dictionary
series_dict = {
    'LASST300000000000003': 'Montana (S)',
    'LASST310000000000003': 'Nebraska (S)',
    'LASST320000000000003': 'Nevada (S)',
    'LASST330000000000003': 'New Hampshire (S)',
    'LASST340000000000003': 'New Jersey (S)',
    'LASST350000000000003': 'New Mexico (S)',
    'LASST360000000000003': 'New York (S)',
    'LASST370000000000003': 'North Carolina (S)',
    'LASST380000000000003': 'North Dakota (S)',
    'LASST390000000000003': 'Ohio (S)',
    'LASST400000000000003': 'Oklahoma (S)',
    'LASST410000000000003': 'Oregon (S)',
    'LASST420000000000003': 'Pennsylvania (S)',
    'LASST440000000000003': 'Rhode Island (S)',
    'LASST450000000000003': 'South Carolina (S)',
    'LASST460000000000003': 'South Dakota (S)',
    'LASST470000000000003': 'Tennessee (S)',
    'LASST480000000000003': 'Texas (S)',
    'LASST490000000000003': 'Utah (S)',
    'LASST500000000000003': 'Vermont (S)',
    'LASST510000000000003': 'Virginia (S)',
    'LASST530000000000003': 'Washington (S)',
    'LASST540000000000003': 'West Virginia (S)',
    'LASST550000000000003': 'Wisconsin (S)',
    'LASST560000000000003': 'Wyoming (S)'}
# Years
dates = ('2018', '2019', '2020', '2021', '2022')

In [ ]:
# JSON request using the URL with the seriesID and dates
headers = {'Content-type': 'application/json'}
# Submit the list of series as data
data3 = json.dumps({
    "seriesid": list(series_dict.keys()),
    "startyear": dates[0],
    "endyear": dates[4]})
# Post request for the data
results3 = requests.post(
    '{}{}'.format(url, key),
    headers=headers,
    data=data3).json()['Results']['series']

In [ ]:
# Date index from first series
date_list = [f"{i['year']}-{i['period'][1:]}-01" for i in results3[0]['data']]
# Empty dataframe to fill with values
unemployment3_df = pd.DataFrame()
# Build a pandas series from the API results, p
for s in results3:
    unemployment3_df[series_dict[s['seriesID']]] = pd.Series(
        index = pd.to_datetime(date_list),
        data = [i['value'] for i in s['data']]
        ).astype(float).iloc[::-1]
# Show last 5 results
unemployment3_df.tail()

# Merge the 3 data frames into one file

In [ ]:
# merge the 2 dataframes for U.S. States
unemployment_states_df = pd.merge(unemployment2_df, unemployment3_df, left_index=True, right_index=True)
unemployment_states_df.head()

In [ ]:
unemployment_complete_df = pd.merge(unemployment1_df, unemployment_states_df, left_index=True, right_index=True)
unemployment_complete_df.head()

In [ ]:
unemployment_complete_df.dtypes

In [ ]:
# Save the DataFrame into a csv file, do not include index
unemployment_complete_df.to_csv("../output/Unemployment_Rates.csv", index=True)

# Additional data points

* Prove negative correlation between Unemployment Rate and CPI
    * CPI - Consumer Price Index is the rate of inflation or an indicator of rising prices in the U.S. Economy
    * Consumer Price Index for All Urban Consumers: All Items in U.S. City Average
    * https://fred.stlouisfed.org/series/CPIAUCSL
* Impact of Covid19 pandemic
    * Using data found on Kaggle.com, show if there is correlation how Covid19 cases in the US impacted unemployment rates
    * https://www.kaggle.com/datasets/antgoldbloom/covid19-data-from-john-hopkins-university?select=CONVENIENT_us_confirmed_cases.csv

CSV Download from the FRED for CPI


In [ ]:
# Open the csv file and create a DataFrame
cpi_df = pd.read_csv("../resource/CPIAUCSL.csv")

# Rename the columns
cpi_df.columns = ["Date", "CPI"]

# Convert the Date into a DateTime format and CPI to float
cpi_df["Date"] = pd.to_datetime(cpi_df["Date"]).dt.normalize()
cpi_df["CPI"]  = cpi_df["CPI"].astype(float)

# Sort on Date and reset index
cpi_df = cpi_df.sort_values("Date", ascending=True)
cpi_df = cpi_df.reset_index(drop=True)

# Display the DataFrame
cpi_df.head()

In [ ]:
# Check the object types
cpi_df.dtypes

CSV Download from Kaggle.com on COVID19 Cases across the U.S.

In [ ]:
# Open the csv file and create a DataFrame
covid19_df = pd.read_csv("../resource/CONVENIENT_us_confirmed_cases.csv")

# Drop the row that contains the county headers since we will be combining the data by State
covid19_df = covid19_df.drop(covid19_df.index[0])

# Rename the Province_State column to Time Period
covid19_df = covid19_df.rename(columns={"Province_State": "Time Period"})

# Convert the Covid19 cases into an integer or float type, and since we do not want to convert all the values, we need to exclude Time Period in the conversion
exclude = ["Time Period"]

# Convert all the data in the DataFrame, excluding the column(s) specified in exclude
covid19_df[covid19_df.columns.difference(exclude)] = covid19_df[covid19_df.columns.difference(exclude)].apply(pd.to_numeric, errors='coerce')

# For any values that did not convert (NaN value), replace with 0
covid19_df = covid19_df.fillna(0)

# Reset the index since we dropped a row earlier
covid19_df = covid19_df.reset_index(drop=True)

# Display the DataFrame
covid19_df.head()

Combine the State County Covid19 cases by State.

The Covid19 file shows the no. of Covid19 cases by day and by state counties. In order to match it up with the Unemployment Rates, we must combine the state county data into each state.

In [ ]:
# To combine the state counties into each state, sum up the values in the columns, axis=1 to only sum per row
covid19_df["Alabama"] = covid19_df.iloc[:, 1:69].sum(axis=1)
covid19_df["Alaska"] = covid19_df.iloc[:, 70:101].sum(axis=1)
covid19_df["Arizona"] = covid19_df.iloc[:, 102:120].sum(axis=1)
covid19_df["Arkansas"] = covid19_df.iloc[:, 121:197].sum(axis=1)
covid19_df["California"] = covid19_df.iloc[:, 198:257].sum(axis=1)
covid19_df["Colorado"] = covid19_df.iloc[:, 258:323].sum(axis=1)
covid19_df["Connecticut"] = covid19_df.iloc[:, 324:333].sum(axis=1)
covid19_df["Delaware"] = covid19_df.iloc[:, 334:338].sum(axis=1)
covid19_df["District of Columbia"] = covid19_df.iloc[:, 340:342].sum(axis=1)
covid19_df["Florida"] = covid19_df.iloc[:, 343:411].sum(axis=1)
covid19_df["Georgia"] = covid19_df.iloc[:, 412:572].sum(axis=1)
covid19_df["Hawaii"] = covid19_df.iloc[:, 575:581].sum(axis=1)
covid19_df["Idaho"] = covid19_df.iloc[:, 582:627].sum(axis=1)
covid19_df["Illinois"] = covid19_df.iloc[:, 628:731].sum(axis=1)
covid19_df["Indiana"] = covid19_df.iloc[:, 732:825].sum(axis=1)
covid19_df["Iowa"] = covid19_df.iloc[:, 826:926].sum(axis=1)
covid19_df["Kansas"] = covid19_df.iloc[:, 927:1033].sum(axis=1)
covid19_df["Kentucky"] = covid19_df.iloc[:, 1034:1155].sum(axis=1)
covid19_df["Louisiana"] = covid19_df.iloc[:, 1156:1221].sum(axis=1)
covid19_df["Maine"] = covid19_df.iloc[:, 1222:1239].sum(axis=1)
covid19_df["Maryland"] = covid19_df.iloc[:, 1240:1265].sum(axis=1)
covid19_df["Massachusetts"] = covid19_df.iloc[:, 1266:1282].sum(axis=1)
covid19_df["Michigan"] = covid19_df.iloc[:, 1283:1369].sum(axis=1)
covid19_df["Minnesota"] = covid19_df.iloc[:, 1370:1458].sum(axis=1)
covid19_df["Mississippi"] = covid19_df.iloc[:, 1459:1542].sum(axis=1)
covid19_df["Missouri"] = covid19_df.iloc[:, 1543:1660].sum(axis=1)
covid19_df["Montana"] = covid19_df.iloc[:, 1661:1718].sum(axis=1)
covid19_df["Nebraska"] = covid19_df.iloc[:, 1719:1813].sum(axis=1)
covid19_df["Nevada"] = covid19_df.iloc[:, 1814:1832].sum(axis=1)
covid19_df["New Hampshire"] = covid19_df.iloc[:, 1833:1844].sum(axis=1)
covid19_df["New Jersey"] = covid19_df.iloc[:, 1845:1867].sum(axis=1)
covid19_df["New Mexico"] = covid19_df.iloc[:, 1868:1902].sum(axis=1)
covid19_df["New York"] = covid19_df.iloc[:, 1903:1966].sum(axis=1)
covid19_df["North Carolina"] = covid19_df.iloc[:, 1967:2068].sum(axis=1)
covid19_df["North Dakota"] = covid19_df.iloc[:, 2069:2123].sum(axis=1)
covid19_df["Ohio"] = covid19_df.iloc[:, 2125:2214].sum(axis=1)
covid19_df["Oklahoma"] = covid19_df.iloc[:, 2215:2293].sum(axis=1)
covid19_df["Oregon"] = covid19_df.iloc[:, 2294:2331].sum(axis=1)
covid19_df["Pennsylvania"] = covid19_df.iloc[:, 2332:2400].sum(axis=1)
covid19_df["Puerto Rico"] = covid19_df.iloc[:, 2401:2480].sum(axis=1)
covid19_df["Rhode Island"] = covid19_df.iloc[:, 2481:2487].sum(axis=1)
covid19_df["South Carolina"] = covid19_df.iloc[:, 2488:2535].sum(axis=1)
covid19_df["South Dakota"] = covid19_df.iloc[:, 2536:2603].sum(axis=1)
covid19_df["Tennessee"] = covid19_df.iloc[:, 2604:2700].sum(axis=1)
covid19_df["Texas"] = covid19_df.iloc[:, 2701:2956].sum(axis=1)
covid19_df["Utah"] = covid19_df.iloc[:, 2957:2993].sum(axis=1)
covid19_df["Vermont"] = covid19_df.iloc[:, 2994:3009].sum(axis=1)
covid19_df["Virginia"] = covid19_df.iloc[:, 3011:3145].sum(axis=1)
covid19_df["Washington"] = covid19_df.iloc[:, 3146:3186].sum(axis=1)
covid19_df["West Virginia"] = covid19_df.iloc[:, 3187:3243].sum(axis=1)
covid19_df["Wisconsin"] = covid19_df.iloc[:, 3244:3317].sum(axis=1)
covid19_df["Wyoming"] = covid19_df.iloc[:, 3318:3342].sum(axis=1)
covid19_df["Total"] = covid19_df.iloc[:, 1:3342].sum(axis=1)


# Create a new column for Year and Month by converting the date column
covid19_df["Time Period"] = pd.to_datetime(covid19_df["Time Period"])
covid19_df["Month"] = covid19_df["Time Period"].dt.month
covid19_df["Year"] = covid19_df["Time Period"].dt.year
covid19_df["Month/Year"] = covid19_df["Month"].astype(str) + "-" + covid19_df["Year"].astype(str)
covid19_df["Date"] = pd.to_datetime(covid19_df["Month/Year"]).dt.normalize()

# Display the DataFrame
covid19_df.head()



In [ ]:
# Create a new Data Frame with the updated data
new_covid19_df = covid19_df[["Date",
                            "Month/Year",
                            "Alabama", 
                            "Alaska",
                            "American Samoa", 
                            "Arizona", 
                            "Arkansas", 
                            "California", 
                            "Colorado", 
                            "Connecticut", 
                            "Delaware",
                            "Diamond Princess", 
                            "District of Columbia",
                            "Florida",
                            "Georgia",
                            "Grand Princess",
                            "Guam",
                            "Hawaii",
                            "Idaho",
                            "Illinois",
                            "Indiana",
                            "Iowa",
                            "Kansas",
                            "Kentucky",
                            "Louisiana",
                            "Maine",
                            "Maryland",
                            "Massachusetts",
                            "Michigan",
                            "Minnesota",
                            "Mississippi",
                            "Missouri",
                            "Montana",
                            "Nebraska",
                            "Nevada",
                            "New Hampshire",
                            "New Jersey",
                            "New Mexico",
                            "New York",
                            "North Carolina",
                            "North Dakota",
                            "Northern Mariana Islands",
                            "Ohio",
                            "Oklahoma",
                            "Oregon",
                            "Pennsylvania",
                            "Puerto Rico",
                            "Rhode Island",
                            "South Carolina",
                            "South Dakota",
                            "Tennessee",
                            "Texas",
                            "Utah",
                            "Vermont",
                            "Virgin Islands",
                            "Virginia",
                            "Washington",
                            "West Virginia",
                            "Wisconsin",
                            "Wyoming",
                            "Total"
                            ]]

# Sort on Date and reset index
new_covid19_df = new_covid19_df.sort_values("Date", ascending=True)
new_covid19_df = new_covid19_df.reset_index(drop=True)

# Save the DataFrame into a csv file, do not include index
new_covid19_df.to_csv("../output/new_covid19.csv", index=False)

# Display results
new_covid19_df.head()

Combine the Covid19 Cases for each State by Date


In [ ]:
# In order to combine monthly Covid19 cases, use the groupby() function
# Sum the Covid19 cases by Date
aggregated = new_covid19_df.groupby("Month/Year").sum()
aggregated = aggregated.reset_index()
 
# Convert into DataFrame before summing up into each State
aggregated_df = pd.DataFrame(aggregated)

# Recreate the date column, insert it in position 1 and sort data by Date
month_year = aggregated_df["Month/Year"]
aggregated_df.insert(1,"Date", month_year)
aggregated_df["Date"] = pd.to_datetime(aggregated_df["Month/Year"]).dt.normalize()
aggregated_df = aggregated_df.sort_values("Date", ascending=True)

# Save the aggregated data into a new file
aggregated_df.to_csv("../output/Aggregated_Covid19.csv", index=False)

# Display results
aggregated_df.head()

Merge CPI data with the Covid Date

In [ ]:
# Merge the unemployment_rates_time_df with cpi_df
merged_covid_cpi_df = pd.merge(aggregated_df, cpi_df, on=["Date", "Date"], how="left")
# Save the merged data into a new file
merged_covid_cpi_df.to_csv("../output/Merged_Covid_CPI.csv", index=False)
# Display results of merge
merged_covid_cpi_df.head()